# G-computation Examples

Python code illustrating the application of the proposed g-computation estimators to a single data set in the context of the case studies described in the paper.

Paul Zivich (2024/12/18)

In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import delicatessen as deli
from delicatessen import MEstimator
from delicatessen.estimating_equations import ee_regression
from delicatessen.utilities import inverse_logit

print("Versions")
print("NumPy:       ", np.__version__)
print("SciPy:       ", sp.__version__)
print("Pandas:      ", pd.__version__)
print("Delicatessen:", deli.__version__)

Versions
NumPy:        1.25.2
SciPy:        1.11.2
Pandas:       1.4.1
Delicatessen: 3.0


## Case 1: Treatment Induced Selection

The following code analyzes an example data set generated using the mechanism described in the paper. Note the true average causal effect is -0.2187.

In [2]:
# Loading in example data
d = pd.read_csv("example1.csv")
d['AW'] = d['A'] * d['W']

# Generating copies of data under interventions
da1 = d.copy()
da1['A'] = 1
da1['AW'] = da1['A'] * da1['W']
da0 = d.copy()
da0['A'] = 0
da0['AW'] = da0['A'] * da0['W']

# NumPy arrays to simplify later data management
y = np.asarray(d['Y'])
a = np.asarray(d['A'])
s = np.asarray(d['S'])
X = np.asarray(d[['I', 'A', 'W', 'AW']])
X1 = np.asarray(da1[['I', 'A', 'W', 'AW']])
X0 = np.asarray(da0[['I', 'A', 'W', 'AW']])

### Standard g-computation

With treatment-induced selection bias, standard g-computation is expected to be biased, as detailed in the paper.

In [3]:
def psi_sg(theta):
    # Subset parameters for easier tracking in code later
    rd, r1, r0 = theta[0:3]
    beta = theta[3:]

    # Outcome nuisance model
    ee_out = ee_regression(beta, X=X, y=y, model='logistic')  # Built-in estimating equation for regression
    ee_out = ee_out * s                                       # Fit model to only those with S=1
    y1hat = inverse_logit(np.dot(X1, beta))                   # Predictions when setting A=1
    y0hat = inverse_logit(np.dot(X0, beta))                   # Predictions when setting A=0

    # Estimating equations for risk parameters
    ee_r1 = y1hat - r1
    ee_r0 = y0hat - r0
    ee_rd = np.ones(y.shape) * (r1 - r0 - rd)

    # Returning the stacked estimating functions
    return np.vstack([ee_rd, ee_r1, ee_r0, ee_out])

In [4]:
inits = [0., 0.5, 0.5, 0., 0., 0., 0., ]
estr = MEstimator(psi_sg, init=inits)
estr.estimate()
ci = estr.confidence_intervals()

In [5]:
print("RD:", estr.theta[0])
print("95% CI:", ci[0, :])

RD: -0.36888570729462733
95% CI: [-0.43955539 -0.29821602]


### Modified g-computation

The modified g-computation algorithm described in the paper is applied here. This version is expected to be unbiased.

In [6]:
def psi_pg(theta):
    # Subset parameters for easier tracking in code later
    rd, r1, r0 = theta[0:3]
    beta = theta[3:]

    # Outcome nuisance model
    ee_out = ee_regression(beta, X=X, y=y, model='logistic')  # Built-in estimating equation for regression
    ee_out = ee_out * s                                       # Fit model to only those with S=1
    y1hat = inverse_logit(np.dot(X1, beta))                   # Predictions when setting A=1
    y0hat = inverse_logit(np.dot(X0, beta))                   # Predictions when setting A=0

    # Estimating equations for risk parameters
    ee_r1 = a*(y1hat - r1)
    ee_r0 = (1-a)*(y0hat - r0)
    ee_rd = np.ones(y.shape) * (r1 - r0 - rd)

    # Returning the stacked estimating functions
    return np.vstack([ee_rd, ee_r1, ee_r0, ee_out])

In [7]:
inits = [0., 0.5, 0.5, 0., 0., 0., 0., ]
estr = MEstimator(psi_pg, init=inits)
estr.estimate()
ci = estr.confidence_intervals()

In [8]:
print("RD:", estr.theta[0])
print("95% CI:", ci[0, :])

RD: -0.25351223589731836
95% CI: [-0.32528575 -0.18173872]


This concludes the first case study.

## Case 2: Confounding and Selection Bias

The following code analyzes an example data set generated using the mechanism described in the paper. Note the true average causal effect is -0.2049.

In [9]:
# Loading in example data
d = pd.read_csv("example2.csv")

# Generating copies of data under interventions
da1 = d.copy()
da1['A'] = 1
da0 = d.copy()
da0['A'] = 0

# NumPy arrays to simplify later data management
y = np.asarray(d['Y'])
a = np.asarray(d['A'])
s = np.asarray(d['S'])

### Standard g-computation: X-only

This first analysis only accounts for the variable related to selection bias. This is expected to be biased due to confounding bias.

In [10]:
X = np.asarray(d[['I', 'A', 'X']])
X1 = np.asarray(da1[['I', 'A', 'X']])
X0 = np.asarray(da0[['I', 'A', 'X']])

In [11]:
# Uses the standard g-computation from above
inits = [0., 0.5, 0.5, 0., 0., 0., ]
estr = MEstimator(psi_sg, init=inits)
estr.estimate()
ci = estr.confidence_intervals()

In [12]:
print("Standard g-computation: X-only")
print("RD:", estr.theta[0])
print("95% CI:", ci[0, :])

Standard g-computation: X-only
RD: -0.14571305869889267
95% CI: [-0.22205721 -0.06936891]


### Standard g-computation: Z-only

This second analysis only accounts for the variable related to confounding bias. This is expected to be biased due to selection bias.

In [13]:
X = np.asarray(d[['I', 'A', 'Z']])
X1 = np.asarray(da1[['I', 'A', 'Z']])
X0 = np.asarray(da0[['I', 'A', 'Z']])

In [14]:
# Uses the standard g-computation from above
inits = [0., 0.5, 0.5, 0., 0., 0., ]
estr = MEstimator(psi_sg, init=inits)
estr.estimate()
ci = estr.confidence_intervals()

In [15]:
print("RD:", estr.theta[0])
print("95% CI:", ci[0, :])

RD: -0.16551817082992823
95% CI: [-0.22742265 -0.10361369]


### Standard g-computation: X & Z

This third analysis only accounts for the variable related to selection and confounding bias. However, bias is still expected to occur due to the M-bias structure.

In [16]:
X = np.asarray(d[['I', 'A', 'X', 'Z']])
X1 = np.asarray(da1[['I', 'A', 'X', 'Z']])
X0 = np.asarray(da0[['I', 'A', 'X', 'Z']])

In [17]:
# Uses the standard g-computation from above
inits = [0., 0.5, 0.5, 0., 0., 0., 0., ]
estr = MEstimator(psi_sg, init=inits)
estr.estimate()
ci = estr.confidence_intervals()

In [18]:
print("RD:", estr.theta[0])
print("95% CI:", ci[0, :])

RD: -0.1575507540776314
95% CI: [-0.22976608 -0.08533542]


### Nested g-computation

This last analysis is using the proposed nested g-computation to account for both selection and confounding bias. This g-computation 

In [19]:
# Design matrix for inner expectation
X = np.asarray(d[['I', 'A', 'X', 'Z']])
X1 = np.asarray(da1[['I', 'A', 'X', 'Z']])
X0 = np.asarray(da0[['I', 'A', 'X', 'Z']])

# Design matrix for outer expectation
W = np.asarray(d[['I', 'A', 'Z']])
W1 = np.asarray(da1[['I', 'A', 'Z']])
W0 = np.asarray(da0[['I', 'A', 'Z']])

In [20]:
def psi_gcomp_nested(theta):
    # Subset parameters for easier tracking in code later
    idX = 3 + X.shape[1]
    idW = idX + W.shape[1]
    rd, r1, r0 = theta[0:3]
    beta = theta[3:idX]
    gamma_1 = theta[idX: idW]
    gamma_0 = theta[idW:]

    # Inner outcome nuisance model
    ee_inner = ee_regression(beta, X=X, y=y, model='logistic')
    ee_inner = ee_inner * s
    y1hat = inverse_logit(np.dot(X1, beta))
    y0hat = inverse_logit(np.dot(X0, beta))

    # Outer outcome nuisance model
    ee_outer1 = ee_regression(gamma_1, X=W, y=y1hat, model='logistic')
    ee_outer0 = ee_regression(gamma_0, X=W, y=y0hat, model='logistic')

    y1hat_outer = inverse_logit(np.dot(W1, gamma_1))
    y0hat_outer = inverse_logit(np.dot(W0, gamma_0))

    # Risk functions
    ee_r1 = y1hat_outer - r1
    ee_r0 = y0hat_outer - r0
    ee_rd = np.ones(y.shape) * (r1 - r0 - rd)

    return np.vstack([ee_rd, ee_r1, ee_r0, ee_inner, ee_outer1, ee_outer0])

In [21]:
inits = [0., 0.5, 0.5, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]
estr = MEstimator(psi_gcomp_nested, init=inits)
estr.estimate()
ci = estr.confidence_intervals()

In [22]:
print("RD:", estr.theta[0])
print("95% CI:", ci[0, :])

RD: -0.18917324702070035
95% CI: [-0.25612398 -0.12222252]


This concludes the second case study.